In [1]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import seaborn as sns
from IPython.display import display
from matplotlib.ticker import MaxNLocator
import random



# === Problem Setup ===
n = 100  # Number of MAFs (tasks)
m = 5  # Number of processing units
K = 5  # Number of energy levels

L_levels = [20, 18, 12, 7, 3]  # Delay per energy level
E_levels = [10, 30, 40, 45, 120]   # Energy consumption per energy class
E_PU_max = [9000] * m  # Maximum energy consumption per processing unit
CPU_capacity = [2000] * m  # CPU resources per processing unit
f_v_levels = [20, 30, 40, 60, 80]  # CPU requirements for each MAF
C_total_max = 12000  # Total system resource capacity
P_idle = [18] * m  # Idle power consumption per PU
T_total = 400  # Total system execution time
P_max = [30] * m  # PU Full-load power consumption
# Task Chains
num_chains = 10
chain_tasks = {i: list(range(i * 10, (i + 1) * 10)) for i in range(num_chains)}
L_max = [200] * num_chains



alpha_1 = 2500
alpha_2 = 10
alpha_pu = 1

params = (
    alpha_1, alpha_2, alpha_pu, E_levels, L_levels, f_v_levels, 
    CPU_capacity, C_total_max, P_idle, P_max, T_total, 
    chain_tasks, L_max, E_PU_max, n, m
)


In [2]:
import random

# 初始化种群
def initialize_population(pop_size, n, m, K):
    population = []
    for _ in range(pop_size):
        individual = {
            "task_pu": [random.randint(0, m - 1) for _ in range(n)],
            "task_level": [random.randint(0, K - 1) for _ in range(n)]
        }
        population.append(individual)
    return population

# 适应度计算
def calculate_fitness(individual, params):
    fitness = 0

    alpha_1, alpha_2, alpha_pu, E_levels, L_levels, f_v_levels, CPU_capacity, \
    C_total_max, P_idle, P_max, T_total, chain_tasks, L_max, E_PU_max, n, m = params

    active_tasks = len([pu for pu in individual["task_pu"] if pu >= 0])
    fitness += alpha_1 * active_tasks

    task_energy = sum(E_levels[level] for level in individual["task_level"])
    fitness -= alpha_2 * task_energy

    pu_energy = 0
    penalty = 0

    total_cpu_usage = sum(f_v_levels[individual["task_level"][i]] for i in range(n))
    if total_cpu_usage > C_total_max:
        penalty += (total_cpu_usage - C_total_max) * 1000

    for j in range(m):
        tasks_on_pu = [i for i, pu in enumerate(individual["task_pu"]) if pu == j]
        execution_time = sum(L_levels[individual["task_level"][i]] for i in tasks_on_pu)
        idle_time = max(0, T_total - execution_time)
        pu_e = P_idle[j] * idle_time + (P_max[j] - P_idle[j]) * execution_time
        pu_energy += pu_e

        cpu_usage = sum(f_v_levels[individual["task_level"][i]] for i in tasks_on_pu)
        if cpu_usage > CPU_capacity[j]:
            penalty += (cpu_usage - CPU_capacity[j]) * 1000

        if pu_e > E_PU_max[j]:
            penalty += (pu_e - E_PU_max[j]) * 1000

    fitness -= alpha_pu * pu_energy

    for chain_id, tasks in chain_tasks.items():
        chain_delay = sum(L_levels[individual["task_level"][i]] for i in tasks) + (len(tasks) - 1)
        if chain_delay > L_max[chain_id]:
            penalty += (chain_delay - L_max[chain_id]) * 1000

    fitness -= penalty

    return fitness

# 选择（轮盘赌选择）
def select_parents(population, fitnesses):
    min_fit = min(fitnesses)
    adjusted_fitnesses = [f - min_fit + 1 for f in fitnesses]
    parents = random.choices(population, weights=adjusted_fitnesses, k=2)
    return parents

# 交叉
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1["task_pu"]) - 1)
    child = {
        "task_pu": parent1["task_pu"][:crossover_point] + parent2["task_pu"][crossover_point:],
        "task_level": parent1["task_level"][:crossover_point] + parent2["task_level"][crossover_point:]
    }
    return child

# 变异
def mutate(individual, m, K, mutation_rate=0.01):
    for i in range(len(individual["task_pu"])):
        if random.random() < mutation_rate:
            individual["task_pu"][i] = random.randint(0, m - 1)
        if random.random() < mutation_rate:
            individual["task_level"][i] = random.randint(0, K - 1)

# 主程序
def genetic_algorithm(n, m, K, generations, pop_size, mutation_rate, params):
    population = initialize_population(pop_size, n, m, K)
    best_fitness = float('-inf')
    best_individual = None

    for gen in range(generations):
        fitnesses = [calculate_fitness(ind, params) for ind in population]
        new_population = []

        max_fit = max(fitnesses)
        if max_fit > best_fitness:
            best_fitness = max_fit
            best_individual = population[fitnesses.index(max_fit)]

        while len(new_population) < pop_size:
            parents = select_parents(population, fitnesses)
            child = crossover(parents[0], parents[1])
            mutate(child, m, K, mutation_rate)
            new_population.append(child)

        population = new_population

        print(f"Generation {gen + 1}: Best Fitness = {best_fitness:.2f}")

    return best_individual, best_fitness


In [4]:
import time

start_time = time.perf_counter()

best_solution, best_fit = genetic_algorithm(
    n=n,
    m=m,
    K=K,
    generations=50,   # 可调整
    pop_size=30,      # 可调整
    mutation_rate=0.02,  # 可微调
    params=params
)

execution_time = time.perf_counter() - start_time

print("\n✅ 遗传算法执行完成！")
print(f"最佳适应度（目标函数值）: {best_fit:.2f}")
print(f"总执行时间: {execution_time:.4f} 秒")


Generation 1: Best Fitness = 178746.00
Generation 2: Best Fitness = 181128.00
Generation 3: Best Fitness = 181128.00
Generation 4: Best Fitness = 182140.00
Generation 5: Best Fitness = 182140.00
Generation 6: Best Fitness = 183084.00
Generation 7: Best Fitness = 183084.00
Generation 8: Best Fitness = 184206.00
Generation 9: Best Fitness = 185998.00
Generation 10: Best Fitness = 186618.00
Generation 11: Best Fitness = 186618.00
Generation 12: Best Fitness = 186618.00
Generation 13: Best Fitness = 187278.00
Generation 14: Best Fitness = 188268.00
Generation 15: Best Fitness = 188806.00
Generation 16: Best Fitness = 188886.00
Generation 17: Best Fitness = 189076.00
Generation 18: Best Fitness = 189636.00
Generation 19: Best Fitness = 190996.00
Generation 20: Best Fitness = 191638.00
Generation 21: Best Fitness = 191638.00
Generation 22: Best Fitness = 192624.00
Generation 23: Best Fitness = 192624.00
Generation 24: Best Fitness = 192624.00
Generation 25: Best Fitness = 193052.00
Generatio

In [5]:
def analyze_solution(individual, params):
    alpha_1, alpha_2, alpha_pu, E_levels, L_levels, f_v_levels, CPU_capacity, \
    C_total_max, P_idle, P_max, T_total, chain_tasks, L_max, E_PU_max, n, m = params

    # 检查PU CPU资源使用情况
    for j in range(m):
        cpu_usage = sum(f_v_levels[individual["task_level"][i]] 
                        for i in range(n) if individual["task_pu"][i] == j)
        print(f"PU-{j} CPU使用情况: {cpu_usage} / {CPU_capacity[j]}")

    # 检查系统资源总使用情况
    total_cpu_usage = sum(f_v_levels[individual["task_level"][i]] for i in range(n))
    print(f"系统总CPU使用情况: {total_cpu_usage} / {C_total_max}")

    # 检查链延迟情况
    for chain_id, tasks in chain_tasks.items():
        chain_delay = sum(L_levels[individual["task_level"][i]] for i in tasks) + (len(tasks) - 1)
        print(f"Chain-{chain_id} 延迟: {chain_delay} / {L_max[chain_id]}")

    # 检查PU能耗限制
    for j in range(m):
        tasks_on_pu = [i for i, pu in enumerate(individual["task_pu"]) if pu == j]
        execution_time = sum(L_levels[individual["task_level"][i]] for i in tasks_on_pu)
        idle_time = max(0, T_total - execution_time)
        pu_energy = P_idle[j] * idle_time + (P_max[j] - P_idle[j]) * execution_time
        print(f"PU-{j} 能耗: {pu_energy} / {E_PU_max[j]}")

# 调用分析函数
print("\n🔍 遗传算法最佳方案的详细分析：")
analyze_solution(best_solution, params)



🔍 遗传算法最佳方案的详细分析：
PU-0 CPU使用情况: 500 / 2000
PU-1 CPU使用情况: 650 / 2000
PU-2 CPU使用情况: 470 / 2000
PU-3 CPU使用情况: 960 / 2000
PU-4 CPU使用情况: 470 / 2000
系统总CPU使用情况: 3050 / 12000
Chain-0 延迟: 147 / 200
Chain-1 延迟: 178 / 200
Chain-2 延迟: 199 / 200
Chain-3 延迟: 163 / 200
Chain-4 延迟: 172 / 200
Chain-5 延迟: 178 / 200
Chain-6 延迟: 180 / 200
Chain-7 延迟: 177 / 200
Chain-8 延迟: 187 / 200
Chain-9 延迟: 190 / 200
PU-0 能耗: 5436 / 9000
PU-1 能耗: 4896 / 9000
PU-2 能耗: 5436 / 9000
PU-3 能耗: 4914 / 9000
PU-4 能耗: 5376 / 9000
